## PyTorch入门

https://pytorch.org/tutorials/beginner/basics/intro.html

In [1]:
# !pip install jupyterlab-language-pack-zh-CN

大多数机器学习工作流程都涉及处理数据、创建模型、优化模型参数以及保存训练好的模型。本教程向您介绍了在 PyTorch 中实现的完整 ML 工作流程，并提供了有关每个概念的更多信息的链接。

我们将使用 FashionMNIST 数据集来训练一个神经网络，该网络可以预测输入图像是否属于以下类别之一：T 恤/上衣、裤子、套头衫、连衣裙、外套、凉鞋、衬衫、运动鞋、包或踝靴。

### 处理数据

PyTorch 有两个处理数据的原语： torch.utils.data.DataLoader和torch.utils.data.Dataset。 

Dataset存储样本及其相应的标签，并DataLoader用可迭代对象包装Dataset。

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch 提供特定领域的库，例如TorchText、 TorchVision和TorchAudio，所有这些库都包含数据集。在本教程中，我们将使用 TorchVision 数据集。

该torchvision.datasets模块包含Dataset许多现实世界视觉数据的对象，如 CIFAR、COCO（完整列表在此处）。

在本教程中，我们使用 **FashionMNIST** 数据集。每个 TorchVision 都Dataset包含两个参数：transform和 target_transform分别用于修改样本和标签。

### 准备数据

In [3]:
# Download training data from open datasets. 下载训练数据集
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.  下载测试数据集
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

Dataset我们将作为参数传递给DataLoader。这将包装我们数据集上的可迭代对象，并支持自动批处理、采样、混洗和多进程数据加载。
这里我们定义批处理大小为 64，即 dataloader 可迭代对象中的每个元素将返回一批 64 个特征和标签。

In [4]:
batch_size = 64

# Create data loaders 创建数据训练/测试加载器


train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### 创建模型

为了在 PyTorch 中定义神经网络，我们创建一个从nn.Module继承的类。我们在函数中定义网络的层__init__，并在函数中指定数据如何通过网络forward。为了加速神经网络中的操作，我们将其移动到加速器， 例如 CUDA、MPS、MTIA 或 XPU。如果当前加速器可用，我们将使用它。否则，我们使用 CPU。

In [5]:
print(torch.cuda.is_available())
# print(f"Using {device} device")

# device = torch.cuda.current_accelerator().type if torch.cuda.is_available() else "cpu"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using '{device}' device")

True
Using 'cuda' device


### 定义模型

In [6]:
# Define Model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 优化模型参数

为了训练模型，我们需要一个损失函数 和一个优化器。

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单次训练循环中，模型对训练数据集（分批输入）进行预测，并反向传播预测误差以调整模型的参数。

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error 计算预测error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation BP算法
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    

我们还根据测试数据集检查模型的性能，以确保它正在学习。

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程需要经过多次迭代（epoch）。在每个 epoch 中，模型都会学习参数以做出更好的预测。我们打印出模型在每个 epoch 中的准确率和损失；我们希望看到准确率在每个 epoch 中增加，损失减少。

In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n -----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
 -----------------------------
loss: 2.311127  [   64/60000]
loss: 2.299029  [ 6464/60000]
loss: 2.283235  [12864/60000]
loss: 2.271208  [19264/60000]
loss: 2.261518  [25664/60000]
loss: 2.232806  [32064/60000]
loss: 2.236875  [38464/60000]
loss: 2.210459  [44864/60000]
loss: 2.206471  [51264/60000]
loss: 2.179847  [57664/60000]
Test Error: 
 Accuracy: 46.4%, Avg loss: 2.174459 

Epoch 2
 -----------------------------
loss: 2.185943  [   64/60000]
loss: 2.175061  [ 6464/60000]
loss: 2.125737  [12864/60000]
loss: 2.136989  [19264/60000]
loss: 2.086734  [25664/60000]
loss: 2.032086  [32064/60000]
loss: 2.060221  [38464/60000]
loss: 1.988787  [44864/60000]
loss: 1.992028  [51264/60000]
loss: 1.927443  [57664/60000]
Test Error: 
 Accuracy: 53.3%, Avg loss: 1.925258 

Epoch 3
 -----------------------------
loss: 1.951802  [   64/60000]
loss: 1.927480  [ 6464/60000]
loss: 1.817685  [12864/60000]
loss: 1.858215  [19264/60000]
loss: 1.736839  [25664/60000]
loss: 1.689824  [32064/60000]

### 保存模型

保存模型的常用方法是序列化内部状态字典（包含模型参数）。

In [11]:
torch.save(model.state_dict(), "models/FashionMNIST_model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### 加载模型

加载模型的过程包括重新创建模型结构和将状态字典加载到其中。

In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("models/FashionMNIST_model.pth", weights_only=True))

<All keys matched successfully>

### 模型预测

现在可以使用该模型进行预测。

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
